In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, explode,col, when, lit, concat_ws, udf, from_json, struct, to_json,window,sum as agg_sum ,count , aggregate,to_date ,dayofweek,hour , minute ,concat
import pandas as pd
from pyspark.sql.pandas.functions import pandas_udf
from pyspark.sql.functions import collect_list, col, when, lit, concat_ws, udf, from_json, struct, to_json,window,sum as agg_sum ,count , aggregate
from pyspark.sql.pandas.functions import pandas_udf, PandasUDFType
from abc import ABC, abstractmethod, abstractclassmethod
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import col, when, lit, concat_ws, udf, from_json, struct, to_json, window, collect_list ,avg
from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType
from pipeline.points import TEHRAN_POINTS ,Point
from pipeline.core import TehranPipeline

from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType ,ArrayType ,MapType ,IntegerType ,Union
spark = SparkSession.builder.getOrCreate()


In [12]:
df = spark.read.csv("./storage/mini_sample_data2.txt",header=True)
df.show()

+--------+---------+---------+--------------------+
|driverId|      lat|     long|                time|
+--------+---------+---------+--------------------+
|       1|35.703686|51.385687|2020-10-20 12:54:...|
|       1|35.703686|51.385687|2020-10-20 12:54:...|
|       1|35.703686|51.385687|2020-10-20 12:54:...|
|       1|35.703686|51.385687|2020-10-20 12:54:...|
|       1|35.686806|51.435012|2020-10-20 12:54:...|
|       1|35.719774|51.360006|2020-10-20 12:54:...|
|       1|35.716914|51.438037|2020-10-20 12:54:...|
|       1|35.706258|51.371695|2020-10-20 12:54:...|
|       1|35.693407| 51.42686|2020-10-20 12:54:...|
|       1|35.668782|51.425666|2020-10-20 12:54:...|
|       1|35.715623|51.376047|2020-10-20 12:54:...|
|     172|35.671891|51.458438|2020-10-20 11:54:...|
|     172|35.728834|51.375421|2020-10-20 11:54:...|
|     113|35.721141|51.390772|2020-10-20 11:54:...|
|     173|35.671891|51.458438|2020-10-20 14:54:...|
|     173|35.728834|51.375421|2020-10-20 14:54:...|
|     173|35

In [6]:
tehran_pipeline=TehranPipeline()

In [11]:
tehran_pipeline.transform(df).show()

+--------+--------------+----------+----------+----+
|driverId|dayOfWeek_Hour|        p1|        p2|edge|
+--------+--------------+----------+----------+----+
|     172|          3_11|  POINT_90|POINT_5954|   1|
|       1|          3_12|POINT_2459| POINT_180|   4|
|       1|          3_12| POINT_180|POINT_4648|   1|
|       1|          3_12|POINT_4648|POINT_4281|   1|
|       1|          3_12|POINT_4281|POINT_2853|   1|
|       1|          3_12|POINT_2853|POINT_1076|   1|
|       1|          3_12|POINT_1076|  POINT_76|   1|
|       1|          3_12|  POINT_76|POINT_4155|   1|
+--------+--------------+----------+----------+----+

